In [1]:
# disperse
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx
import requests

secret = 61740721216174072121
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.005

response = requests.get('https://testnet.blockexplorer.com/api/addr/{}/utxo'.format(my_address))
utxos = response.json()

student_addresses = [
    'mz1K5YbtyXa6kBvYJXhpq7hukcQQ3kuFCJ'
]
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['amount']
    # create a new tx input
    tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []

for target_address in student_addresses:
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount*100000000)
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_amount = total - target_amount*len(student_addresses) - fee

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv, SIGHASH_ALL)

# SANITY CHECK: fee is reasonable
if tx_obj.fee(testnet=True) > 0.05*100000000 or tx_obj.fee(testnet=True) <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

010000000382887b37aa36414d17d5fc4ef98a449e83d310eff38db9d3015e493c8cd8fc97000000006a47304402202a120a185a74519177f68588658c91b96e2ac7267dd0123fbc85d63090e55f51022030805189c80fd174bfdc461cca13d0ed271e607f4d65fa6d68ac860df5603ace012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff3d85fdc292791ae232a70b4e00ae92385a5a133d62bc4b1cb4b84888f373af7d000000006a47304402204bf6e39dd65e505b2371e7f44521d32c6e75ed8018d28063882e7e11081dcffb02205e100df2f2d955f12553abc8089cb8dac23836a40ef8eb871beb7fdcf730ce76012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff8b966d8d367d69a3bc4bad70c4074531bc68a7df2e14d6c7167976b6a03165f0110000006a4730440220259b3f2ade55c97893cb562e51b71bc4e1da8f240fa90daea287ef5b96cc0cdf02203de71c74931d9b15d35ea7f32da7f4962fcfa5526d8784467e3d0ed2db578690012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff0280969800000000001976a914cacf42cc55e41094b8aeec6aee047fb03d6a6e9d88acbb18482f000000001976a9149e153ce5de3689